In [1]:
# OBJETIVO:
# Dado um número de acórdão precedente de súmula, originado do dataset "sumulas" criado
# pelo notebook _1_scraping_sumulas, baixar o respectivo arquivo PDF do sítio do CARF.

In [1]:
#!pip install beautifulsoup4
import os
import pandas as pd
import re
import requests
import sys
import traceback
import warnings
from bs4 import BeautifulSoup
from importlib.metadata import version

In [2]:
version('beautifulsoup4')

'4.9.3'

In [3]:
# carrega o dataset intermediário "sumulas" criado pelo notebook "1-scraping-sumulas"
sumulas = pd.read_csv('datasets/sumulas.csv')
sumulas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sumula       158 non-null    int64 
 1   Enunciado    158 non-null    object
 2   Precedentes  158 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.8+ KB


In [4]:
# obtém o conjunto de números acórdãos a baixar

# a coluna "Precedentes" contém números concatenados por espaço,
# p. ex., "3201-000.007 3102-002.198 9303-006.331"
nums = [num for text in sumulas['Precedentes'].values for num in text.split()]
print('Total de precedentes:', len(nums))
# remove o '.' do número em razão de variações de grafia
nums = [text.replace('.', '') for text in nums]
# identifica acórdãos precedentes de duas súmulas
nums_2_sum = set([num for num in nums if nums.count(num) == 2])
print('Precedentes de duas súmulas:', len(nums_2_sum), nums_2_sum)
# identifica acórdãos precedentes de mais de duas súmulas
nums_mais_2_sum = set([num for num in nums if nums.count(num) > 2])
print('Precedentes de mais de duas súmulas:', len(nums_mais_2_sum), nums_mais_2_sum)
# remove duplicidades
nums = set(nums)
print('Total de precedentes a baixar:', len(nums))

Total de precedentes: 1198
Precedentes de duas súmulas: 16 {'9303-005293', '195-00008', '106-16727', '9101-002245', '3102-002141', '102-46574', '106-15476', '9101-003376', '105-14173', '108-09286', '1402-00213', '103-21239', '106-17254', '103-21225', '108-09592', '105-16365'}
Precedentes de mais de duas súmulas: 0 set()
Total de precedentes a baixar: 1182


In [5]:
# função para extrair o valor da tag <input>, id=javax.faces.ViewState
def get_view_state(resp):
    soup = BeautifulSoup(resp.text, 'html.parser')
    return soup.select('input[id="javax.faces.ViewState"]')[0]['value']

In [11]:
# função para baixar o arquivo PDF de um acórdão, dado seu número
def baixa_acordao(num, path_to_pdf):
    print('Processando acórdão nº', num)
    
    # abre a página "Pesquisa de Acórdãos" do menu esquerdo "Jurisprudência", "Acórdãos CARF"
    url_base = 'https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/'
    url = url_base + 'consultarJurisprudenciaCarf.jsf'
    with warnings.catch_warnings():
        # suprime o warning gerado pelo parâmetro 'verify=False' em 'requests.get'
        warnings.simplefilter('ignore')
        resp = requests.get(url, verify=False)
    resp.raise_for_status()
    cookies = resp.cookies
    
    # submete pesquisa por número de acórdão
    payload = { 'AJAXREQUEST': '_viewRoot', 'consultaJurisprudenciaForm': 'consultaJurisprudenciaForm',
           'campo_pesquisa1':'3', 'valor_pesquisa1': num, 'campo_pesquisa2':'1', 'campo_pesquisa3':'1',
           'j_id51': 'j_id51', 'javax.faces.ViewState': get_view_state(resp) }
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        resp = requests.post(url, data=payload, cookies=cookies, verify=False)
    resp.raise_for_status()
    # lança exceção se houver mensagem de erro (nenhum registro) ou mais de um resultado
    soup = BeautifulSoup(resp.text, 'html.parser')
    if soup.select('dt.mensagem_erro'):
        raise ValueError('Nenhum registro encontrado')
    else:
        span_hits = soup.find_all('span', text=re.compile(r'Acórdãos Encontrados'))[0]
        hits = int(span_hits.find_next_sibling('span').text)
        if hits > 1:
            raise ValueError('Mais de um registro encontrado:', hits)
     
    # clica o link do número do acórdão
    url = url_base + 'listaJurisprudenciaCarf.jsf'
    payload = { 'tblJurisprudencia:0:numDecisao': 'tblJurisprudencia:0:numDecisao',
           'formAcordaos': 'formAcordaos', 'javax.faces.ViewState': get_view_state(resp) }
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        resp = requests.post(url, data=payload, cookies=cookies, verify=False)
    resp.raise_for_status()
    
    # clica o ícone "Anexos" ao pé da tabela e faz o download do arquivo PDF
    url = url_base + 'listaJurisprudencia.jsf'
    soup = BeautifulSoup(resp.text, 'html.parser')
    id = soup.select('a[id^="formAcordaos:j_id"]')[0]['id']
    payload = { 'formAcordaos': 'formAcordaos', 'formAcordaos:_idcl': id,
           'javax.faces.ViewState': get_view_state(resp) }
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        resp = requests.post(url, data=payload, cookies=cookies, verify=False)
    resp.raise_for_status()
    if resp.headers['Content-Type'] != 'application/pdf':
        raise ValueError('Resposta inesperada: conteúdo não é um PDF')
    # salva o arquivo PDF tendo por nome o número do acórdão
    filename = path_to_pdf + num + '.pdf'
    with open(filename, 'wb') as f:
        f.write(resp.content)  

In [12]:
# baixa os arquivos dos acórdãos para uma pasta local
path_to_pdf = 'pdf/'
os.makedirs(path_to_pdf, exist_ok=True)     

for num in nums:
    try:
        baixa_acordao(num, path_to_pdf)
    except Exception:
        print(traceback.format_exc(), file=sys.stderr)

Processando acórdão nº 106-14971
Processando acórdão nº 3102-002020
Processando acórdão nº 101-95884
Processando acórdão nº 302-37892
Processando acórdão nº 104-22117
Processando acórdão nº 201-73615
Processando acórdão nº 1302-000393


Traceback (most recent call last):
  File "<ipython-input-12-ce7f870e485c>", line 7, in <module>
    baixa_acordao(num, path_to_pdf)
  File "<ipython-input-11-cc788343a120>", line 31, in baixa_acordao
    raise ValueError('Mais de um registro encontrado:', hits)
ValueError: ('Mais de um registro encontrado:', 2)



Processando acórdão nº 107-09410
Processando acórdão nº 104-20812
Processando acórdão nº 9101-003387
Processando acórdão nº 1401-002833
Processando acórdão nº 2402-006646
Processando acórdão nº 9101-01037
Processando acórdão nº 9303-007880
Processando acórdão nº 2801-00239


Traceback (most recent call last):
  File "<ipython-input-12-ce7f870e485c>", line 7, in <module>
    baixa_acordao(num, path_to_pdf)
  File "<ipython-input-11-cc788343a120>", line 31, in baixa_acordao
    raise ValueError('Mais de um registro encontrado:', hits)
ValueError: ('Mais de um registro encontrado:', 33)



Processando acórdão nº 9303-002458
Processando acórdão nº 107-09230
Processando acórdão nº 9202-003401
Processando acórdão nº 9303-003141
Processando acórdão nº 3403-002701
Processando acórdão nº 1302-001164
Processando acórdão nº 04-00627


...

Processando acórdão nº 9303-005436
Processando acórdão nº 1102-00351
Processando acórdão nº 101-88969
Processando acórdão nº 105-16138
Processando acórdão nº 9101-01191
Processando acórdão nº 1402-002815
Processando acórdão nº 3101-001174
Processando acórdão nº 202-11274
Processando acórdão nº 2202-002528
Processando acórdão nº 105-14637
Processando acórdão nº 201-77198
Processando acórdão nº 3101-01072
Processando acórdão nº 2403-001742
Processando acórdão nº 9202-003067
Processando acórdão nº 1401-003024
Processando acórdão nº 202-14254
Processando acórdão nº 2401-02358
Processando acórdão nº 9303-004155
Processando acórdão nº 2301-005


Traceback (most recent call last):
  File "<ipython-input-12-ce7f870e485c>", line 7, in <module>
    baixa_acordao(num, path_to_pdf)
  File "<ipython-input-11-cc788343a120>", line 31, in baixa_acordao
    raise ValueError('Mais de um registro encontrado:', hits)
ValueError: ('Mais de um registro encontrado:', 3372)



Processando acórdão nº 9202-006006
Processando acórdão nº 9101-002484
Processando acórdão nº 303-31479
Processando acórdão nº 102-48290
Processando acórdão nº 9101-002993
Processando acórdão nº 202-16437
Processando acórdão nº 106-16788
Processando acórdão nº 101-94812
Processando acórdão nº 9303-005886
Processando acórdão nº 301-31414
Processando acórdão nº 3402-002252


Traceback (most recent call last):
  File "<ipython-input-12-ce7f870e485c>", line 7, in <module>
    baixa_acordao(num, path_to_pdf)
  File "<ipython-input-11-cc788343a120>", line 31, in baixa_acordao
    raise ValueError('Mais de um registro encontrado:', hits)
ValueError: ('Mais de um registro encontrado:', 2)



Processando acórdão nº 2401-003400
Processando acórdão nº 204-03122
